# Análise de Erros dos Classificadores

**Execute este notebook APÓS rodar `main.py`**

Aqui você vai:
- Ver onde cada modelo erra
- Identificar padrões nos erros
- Encontrar casos onde modelos simples batem modelos complexos
- Entender quais textos são difíceis de classificar

In [ ]:
import sys
sys.path.append('..')

import pickle
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
from collections import Counter

%matplotlib inline
plt.rcParams['figure.figsize'] = (14, 8)
sns.set_style('whitegrid')

## 1. Carregar Dados e Predições

Se você rodou `main.py`, as predições foram salvas em `results/`

In [ ]:
results_dir = Path('../results')

# Verificar se resultados existem
if not results_dir.exists():
    print("⚠️ Pasta 'results/' não encontrada!")
    print("Execute 'python main.py' primeiro.")
    sys.exit()

# Carregar predições salvas (você precisará modificar main.py para salvá-las)
try:
    with open(results_dir / 'predictions.pkl', 'rb') as f:
        data = pickle.load(f)
    
    X_test = data['X_test']
    y_true = data['y_true']
    y_pred_embedding = data['y_pred_embedding']
    y_pred_finetuned = data['y_pred_finetuned']
    y_pred_llm = data['y_pred_llm']
    id2label = data['id2label']
    
    print("✓ Predições carregadas com sucesso!")
    print(f"\nTotal de amostras de teste: {len(X_test)}")
    
except FileNotFoundError:
    print("❌ Arquivo 'predictions.pkl' não encontrado!")
    print("\n⚠️ IMPORTANTE: Você precisa modificar main.py para salvar as predições.")
    print("\nAdicione no final da função main():")
    print("""
    # Salvar predições para análise de erros
    predictions_data = {
        'X_test': X_test,
        'y_true': y_test,
        'y_pred_embedding': embedding_clf.predict(X_test),
        'y_pred_finetuned': finetuned_clf.predict(X_test),
        'y_pred_llm': llm_clf.predict(X_test),
        'id2label': id2label
    }
    
    import pickle
    with open(config.RESULTS_DIR / 'predictions.pkl', 'wb') as f:
        pickle.dump(predictions_data, f)
    """)
    sys.exit()

## 2. Criar DataFrame de Análise

In [ ]:
# Criar DataFrame com todas as informações
df_analysis = pd.DataFrame({
    'text': X_test,
    'true_label': y_true,
    'true_label_name': [id2label[label] for label in y_true],
    'pred_embedding': y_pred_embedding,
    'pred_finetuned': y_pred_finetuned,
    'pred_llm': y_pred_llm,
    'pred_embedding_name': [id2label[label] for label in y_pred_embedding],
    'pred_finetuned_name': [id2label[label] for label in y_pred_finetuned],
    'pred_llm_name': [id2label[label] for label in y_pred_llm],
})

# Adicionar colunas de acerto/erro
df_analysis['correct_embedding'] = df_analysis['true_label'] == df_analysis['pred_embedding']
df_analysis['correct_finetuned'] = df_analysis['true_label'] == df_analysis['pred_finetuned']
df_analysis['correct_llm'] = df_analysis['true_label'] == df_analysis['pred_llm']

# Contar quantos modelos acertaram
df_analysis['num_correct'] = (
    df_analysis['correct_embedding'].astype(int) +
    df_analysis['correct_finetuned'].astype(int) +
    df_analysis['correct_llm'].astype(int)
)

print("DataFrame de análise criado!")
df_analysis.head()

## 3. Visão Geral dos Erros

In [ ]:
# Resumo de acertos por modelo
print("=" * 60)
print("RESUMO DE ACERTOS")
print("=" * 60)

for model in ['embedding', 'finetuned', 'llm']:
    correct = df_analysis[f'correct_{model}'].sum()
    total = len(df_analysis)
    accuracy = correct / total * 100
    print(f"{model.upper():<15}: {correct:>4}/{total} ({accuracy:>5.1f}%)")

print("\n" + "=" * 60)
print("CONSENSO DOS MODELOS")
print("=" * 60)

consensus_counts = df_analysis['num_correct'].value_counts().sort_index(ascending=False)

for num_correct, count in consensus_counts.items():
    pct = count / len(df_analysis) * 100
    if num_correct == 3:
        label = "Todos acertaram"
    elif num_correct == 2:
        label = "2 modelos acertaram"
    elif num_correct == 1:
        label = "Apenas 1 acertou"
    else:
        label = "NENHUM acertou ❌"
    
    print(f"{label:<25}: {count:>4} amostras ({pct:>5.1f}%)")

## 4. Casos Onde NENHUM Modelo Acertou

Estes são os casos mais difíceis!

In [ ]:
# Casos onde todos erraram
all_wrong = df_analysis[df_analysis['num_correct'] == 0]

print(f"Total de casos onde TODOS erraram: {len(all_wrong)}")
print("\n" + "=" * 80)

# Mostrar alguns exemplos
num_examples = min(10, len(all_wrong))

for i, (idx, row) in enumerate(all_wrong.head(num_examples).iterrows(), 1):
    print(f"\n{i}. Texto: {row['text'][:100]}...")
    print(f"   ✓ Verdadeiro: {row['true_label_name']}")
    print(f"   ✗ Embedding:  {row['pred_embedding_name']}")
    print(f"   ✗ Fine-tuned: {row['pred_finetuned_name']}")
    print(f"   ✗ LLM:        {row['pred_llm_name']}")
    print("-" * 80)

### 🔍 Análise: Por que todos erraram?

**Possíveis razões:**

1. **Ambiguidade genuína**: O texto pode expressar múltiplas emoções
2. **Label errado no dataset**: Pode ser erro de anotação
3. **Contexto insuficiente**: Falta informação para determinar a emoção
4. **Viés do dataset**: Padrões não representativos no treino
5. **Classe minoritária**: Classes raras (ex: surprise) são mais difíceis

## 5. Casos Onde Apenas UM Modelo Acertou

Vamos ver qual modelo é melhor em casos difíceis

In [ ]:
# Casos onde apenas 1 acertou
only_one_correct = df_analysis[df_analysis['num_correct'] == 1]

print(f"Total de casos onde APENAS 1 modelo acertou: {len(only_one_correct)}")
print("\nQuem acertou quando os outros erraram?\n")

# Contar vitórias individuais
embedding_alone = only_one_correct[
    only_one_correct['correct_embedding'] & 
    ~only_one_correct['correct_finetuned'] & 
    ~only_one_correct['correct_llm']
]

finetuned_alone = only_one_correct[
    ~only_one_correct['correct_embedding'] & 
    only_one_correct['correct_finetuned'] & 
    ~only_one_correct['correct_llm']
]

llm_alone = only_one_correct[
    ~only_one_correct['correct_embedding'] & 
    ~only_one_correct['correct_finetuned'] & 
    only_one_correct['correct_llm']
]

print(f"Embedding sozinho:  {len(embedding_alone):>4} casos")
print(f"Fine-tuned sozinho: {len(finetuned_alone):>4} casos")
print(f"LLM sozinho:        {len(llm_alone):>4} casos")

# Gráfico de pizza
labels = ['Embedding', 'Fine-tuned', 'LLM']
sizes = [len(embedding_alone), len(finetuned_alone), len(llm_alone)]
colors = ['#FF9999', '#66B3FF', '#99FF99']

plt.figure(figsize=(10, 6))
plt.pie(sizes, labels=labels, colors=colors, autopct='%1.1f%%', startangle=90)
plt.title('Vitórias Individuais: Quem acertou quando os outros erraram?')
plt.axis('equal')
plt.show()

### Exemplos de Vitórias Individuais

In [ ]:
# Mostrar exemplos onde cada modelo foi o único a acertar
datasets = [
    ('EMBEDDING', embedding_alone),
    ('FINE-TUNED', finetuned_alone),
    ('LLM', llm_alone)
]

for model_name, df_subset in datasets:
    if len(df_subset) == 0:
        continue
    
    print("\n" + "=" * 80)
    print(f"Casos onde APENAS {model_name} acertou")
    print("=" * 80)
    
    num_examples = min(3, len(df_subset))
    
    for i, (idx, row) in enumerate(df_subset.head(num_examples).iterrows(), 1):
        print(f"\n{i}. Texto: {row['text'][:100]}...")
        print(f"   ✓ Verdadeiro:  {row['true_label_name']}")
        print(f"   {'✓' if row['correct_embedding'] else '✗'} Embedding:   {row['pred_embedding_name']}")
        print(f"   {'✓' if row['correct_finetuned'] else '✗'} Fine-tuned:  {row['pred_finetuned_name']}")
        print(f"   {'✓' if row['correct_llm'] else '✗'} LLM:         {row['pred_llm_name']}")
        print("-" * 80)

## 6. Modelo Simples vs Complexo

Casos onde Embedding (simples) acertou mas Fine-tuned (complexo) errou

In [ ]:
# Casos onde embedding acertou MAS finetuned errou
simple_beats_complex = df_analysis[
    df_analysis['correct_embedding'] & ~df_analysis['correct_finetuned']
]

# Casos onde finetuned acertou MAS embedding errou
complex_beats_simple = df_analysis[
    ~df_analysis['correct_embedding'] & df_analysis['correct_finetuned']
]

print("Comparação: Embedding vs Fine-tuned")
print("=" * 60)
print(f"Embedding acertou, Fine-tuned errou:  {len(simple_beats_complex):>4} casos")
print(f"Fine-tuned acertou, Embedding errou:  {len(complex_beats_simple):>4} casos")
print(f"\nRazão (Complex/Simple):               {len(complex_beats_simple) / max(len(simple_beats_complex), 1):.2f}x")

# Mostrar exemplos onde simples bateu complexo
if len(simple_beats_complex) > 0:
    print("\n" + "=" * 80)
    print("Exemplos: Embedding (simples) acertou mas Fine-tuned (complexo) errou")
    print("=" * 80)
    
    for i, (idx, row) in enumerate(simple_beats_complex.head(5).iterrows(), 1):
        print(f"\n{i}. Texto: {row['text'][:100]}...")
        print(f"   ✓ Verdadeiro:  {row['true_label_name']}")
        print(f"   ✓ Embedding:   {row['pred_embedding_name']} (ACERTOU)")
        print(f"   ✗ Fine-tuned:  {row['pred_finetuned_name']} (ERROU)")
        print("-" * 80)

### 💡 Insight: Quando o modelo simples ganha?

**Possíveis razões:**

1. **Overfitting**: Fine-tuned pode ter decorado padrões específicos do treino
2. **Generalização**: Embeddings pré-treinados têm conhecimento mais geral
3. **Dataset pequeno**: Fine-tuning com poucos dados pode não funcionar bem
4. **Hiperparâmetros**: Learning rate, epochs podem estar mal configurados

## 7. Análise de Erros por Classe

Quais classes cada modelo tem mais dificuldade?

In [ ]:
# Criar matriz de erros por classe
error_by_class = []

for label_id, label_name in id2label.items():
    df_class = df_analysis[df_analysis['true_label'] == label_id]
    
    if len(df_class) == 0:
        continue
    
    total = len(df_class)
    
    error_by_class.append({
        'Classe': label_name,
        'Total': total,
        'Embedding Erros': (~df_class['correct_embedding']).sum(),
        'Fine-tuned Erros': (~df_class['correct_finetuned']).sum(),
        'LLM Erros': (~df_class['correct_llm']).sum(),
        'Embedding Erro %': (~df_class['correct_embedding']).sum() / total * 100,
        'Fine-tuned Erro %': (~df_class['correct_finetuned']).sum() / total * 100,
        'LLM Erro %': (~df_class['correct_llm']).sum() / total * 100,
    })

df_errors = pd.DataFrame(error_by_class)

print("Taxa de Erro por Classe (%)")
print("=" * 80)
print(df_errors[['Classe', 'Total', 'Embedding Erro %', 'Fine-tuned Erro %', 'LLM Erro %']].to_string(index=False))

# Gráfico de barras
fig, ax = plt.subplots(figsize=(12, 6))

x = np.arange(len(df_errors))
width = 0.25

ax.bar(x - width, df_errors['Embedding Erro %'], width, label='Embedding', color='#FF9999')
ax.bar(x, df_errors['Fine-tuned Erro %'], width, label='Fine-tuned', color='#66B3FF')
ax.bar(x + width, df_errors['LLM Erro %'], width, label='LLM', color='#99FF99')

ax.set_xlabel('Classe')
ax.set_ylabel('Taxa de Erro (%)')
ax.set_title('Taxa de Erro por Classe')
ax.set_xticks(x)
ax.set_xticklabels(df_errors['Classe'], rotation=45, ha='right')
ax.legend()
ax.grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.show()

## 8. Confusões Mais Comuns

Quais pares de classes são mais confundidos?

In [ ]:
def get_top_confusions(y_true, y_pred, id2label, top_n=5):
    """Retorna as confusões mais comuns (exceto acertos)"""
    confusions = []
    
    for true_label, pred_label in zip(y_true, y_pred):
        if true_label != pred_label:  # Apenas erros
            confusions.append((id2label[true_label], id2label[pred_label]))
    
    counter = Counter(confusions)
    return counter.most_common(top_n)

# Confusões mais comuns de cada modelo
models = [
    ('Embedding', y_pred_embedding),
    ('Fine-tuned', y_pred_finetuned),
    ('LLM', y_pred_llm)
]

for model_name, y_pred in models:
    print("\n" + "=" * 80)
    print(f"Top 5 Confusões: {model_name}")
    print("=" * 80)
    
    top_confusions = get_top_confusions(y_true, y_pred, id2label)
    
    for i, ((true_class, pred_class), count) in enumerate(top_confusions, 1):
        print(f"{i}. '{true_class}' → confundido com '{pred_class}': {count} vezes")

### 🔍 Análise de Confusões

**Confusões esperadas:**

- **sadness ↔ fear**: Ambas são emoções negativas
- **joy ↔ love**: Ambas são emoções positivas
- **anger ↔ fear**: Contextos de ameaça
- **surprise ↔ joy**: Surpresas geralmente positivas

**Se vir confusões inesperadas:**
- Pode indicar problema nos dados de treino
- Ou ambiguidade genuína nas labels

## 9. Exemplos de Confusões Específicas

Vamos ver textos reais das confusões mais comuns

In [ ]:
def show_confusion_examples(df, true_class, pred_class, model_col, num_examples=3):
    """Mostra exemplos de uma confusão específica"""
    mask = (df['true_label_name'] == true_class) & (df[model_col] == pred_class)
    examples = df[mask].head(num_examples)
    
    if len(examples) == 0:
        print("   (Nenhum exemplo encontrado)")
        return
    
    for i, (idx, row) in enumerate(examples.iterrows(), 1):
        print(f"\n   {i}. {row['text'][:100]}...")

# Pegar confusão mais comum do fine-tuned
top_confusion = get_top_confusions(y_true, y_pred_finetuned, id2label, top_n=1)[0]
true_class, pred_class = top_confusion[0]

print("=" * 80)
print(f"Exemplos: '{true_class}' confundido com '{pred_class}' (Fine-tuned)")
print("=" * 80)

show_confusion_examples(df_analysis, true_class, pred_class, 'pred_finetuned_name', num_examples=5)

## 10. Recomendações Baseadas nos Erros

### 🎯 Como Melhorar os Modelos?

**Baseado na análise de erros:**

1. **Se muitos erros nas mesmas classes:**
   - Coletar mais dados dessas classes
   - Aumentar peso dessas classes no treinamento
   - Usar data augmentation

2. **Se confusões entre classes específicas:**
   - Revisar as labels do dataset (pode ter erro de anotação)
   - Adicionar features que diferenciem essas classes
   - Considerar fusão de classes muito similares

3. **Se modelo simples bate complexo:**
   - Reduzir learning rate
   - Aumentar regularização (weight decay)
   - Treinar por mais épocas
   - Coletar mais dados de treino

4. **Se todos os modelos erram nos mesmos casos:**
   - Problema pode ser no dataset (labels ruins)
   - Ou são casos genuinamente ambíguos
   - Considerar rejeitar predições com baixa confiança

### 📊 Próximos Passos

1. **Analise as confusões** mais comuns de cada modelo
2. **Leia os exemplos** onde todos erraram
3. **Compare com seus objetivos**: Qual tipo de erro é mais crítico?
4. **Ajuste o modelo** baseado nos insights
5. **Re-treine e compare** novamente

## 11. Exportar Análise

Salvar análise para referência futura

In [ ]:
# Salvar DataFrame completo
output_path = results_dir / 'error_analysis.csv'
df_analysis.to_csv(output_path, index=False)
print(f"✓ Análise salva em: {output_path}")

# Salvar resumo de erros por classe
error_summary_path = results_dir / 'error_summary_by_class.csv'
df_errors.to_csv(error_summary_path, index=False)
print(f"✓ Resumo de erros salvo em: {error_summary_path}")